The task at hand is to categorize songs into certain categories (trap, rap, pop etc) based off of given values such as danceability, acousticness and other metrics that can be readily obtained via Spotify's organize playlist. <br>
The datasets were obtained from https://www.kaggle.com/iamsumat/spotify-top-2000s-mega-dataset and https://www.kaggle.com/leonardopena/top-spotify-songs-from-20102019-by-year?select=top10s.csv

Two datasets are being used to increase the amount of data available. Note that both sets of data use the same metrics available off of Spotify's organize playlist feature. These metrics are as follows:
1. **Genre** - the genre of the track
2. **Year** - the release year of the recording. Note that due to vagaries of releases, re-releases, re-issues and general madness, sometimes the release years are not what you'd expect.
3. **Added** - the earliest date you added the track to your collection.
4. **Beats Per Minute** (BPM) - The tempo of the song.
5. **Energy** - The energy of a song - the higher the value, the more energtic. song
6. **Danceability** - The higher the value, the easier it is to dance to this song.
7. **Loudness** (dB) - The higher the value, the louder the song.
8. **Liveness** - The higher the value, the more likely the song is a live recording.
9. **Valence** - The higher the value, the more positive mood for the song.
10. **Length** - The duration of the song.
11. **Acousticness** - The higher the value the more acoustic the song is.
12. **Speechiness** - The higher the value the more spoken word the song contains.
13. **Popularity** - The higher the value the more popular the song is.
14. **Duration** - The length of the song.

<h2> Obtaining Data

In [ ]:
from pandas import read_csv
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import numpy as np

In [ ]:
df_2000 = pd.read_csv("../input/spotify-top-2000s-mega-dataset/Spotify-2000.csv")
df_top10s = pd.read_csv("../input/top-spotify-songs-from-20102019-by-year/top10s.csv", engine='python') # the engine needs to be changed otherwise UTF-8 error occurs
df_2000.head()

In [ ]:
df_top10s.head()

In [ ]:
df_2000.info()

In [ ]:
df_top10s.info()

In [ ]:
len(df_2000["Top Genre"].unique()), len(df_top10s["top genre"].unique())

In [ ]:
df_2000["Top Genre"].value_counts(), df_top10s["top genre"].value_counts()

From a quick observation, we can see certain things about the data we are dealing with:<br>
1. Both datasets use the same metrics, albeit with different column names so we will have to change those names. Fortunately, the same order is kept for both datasets
2. Our target label is the top genre category, however there is a very large number of genre types, so many in fact that our model will be very inaccurate and inefficient if we attempt to label all categories as they are, especially since a large number of categories contain a single song only. 
3. The two datasets do not share the same # of genres.
4. We don't care about the artist, title, index/Unamed: 0, and year. I simply don't think these would have a strong connection with the genre. (Arguably, artist would have a connection to the music genre, but I personally think there is too much variance and it wouldn't be a consistent pattern)

<h2> Data Preparation

Dropping off all unecessary columns

In [ ]:
df_top10s.info()

In [ ]:
df_2000.drop(columns = ['Index', 'Title', 'Artist', 'Year'], inplace = True)
df_top10s.drop(columns = ['Unnamed: 0', 'title', 'artist', 'year'], inplace = True)

Now both datasets have the same # of columsn in the same order, so we can now join them together after renaming

In [ ]:
df_top10s.columns = df_2000.columns # setting column names as each other
df = df_2000.append(df_top10s, ignore_index = True)

both X_train and X_test have several values that are string type numbers, separated with a comma. We have to get rid of these commas then turn them into floats

In [ ]:
attributes = df.columns[1:]
for attribute in attributes:
    temp = df[attribute]
    for instance in range(len(temp)):
        if(type(temp[instance]) == str):
            df[attribute][instance] = float(temp[instance].replace(',',''))
# check data types using df.dtype

<h3> Splitting Genre

Now that we have obtained our full dataset, we need a way to split the top genres. Two methods will be explored: <br>
**Method 1** All related songs of a specific category will be placed in a broader category (i.e. celtic pop and indie pop will be placed under the larger theme of pop). The main assumption using this method is that there is minimal difference between differing types of a similar music genre. This will be multiclass classification <br>
**Method 2** Songs will have their genres split by space and multilabel classification will be used. <br>
**NOTE**: Method 2 has been pushed to the bottom after Method 1 is completed to prevent interference and distraction from cross coding

In [ ]:
# first extracting the genre columns
# getting rid of white spaces and turning it all into lower cases
genre = (df["Top Genre"].str.strip()).str.lower()

<h4> Method 1

In [ ]:
# function to split the genre column
def genre_splitter(genre):
    result = genre.copy()
    result = result.str.split(" ",1)
    for i in range(len(result)):
        if (len(result[i]) > 1):
            result[i] = [result[i][1]]
    return result.str.join('')

In [ ]:
# loop until the genre cannot be split any further
genre_m1 = genre.copy()
while(max((genre_m1.str.split(" ", 1)).str.len()) > 1):
    genre_m1 = genre_splitter(genre_m1)

In [ ]:
len(genre_m1.unique())

We reduced our target label by over 50%. Let's take a quick look at our new shortened labels

In [ ]:
genre_m1.value_counts()

Note that there are certain music genres that have a single value. These genres would make our model inefficient, since it does not have data to work off of, so these values and corresponding rows in the original dataframe will be taken out. Putting all of these instances into a broader "Other" category is another potential solution, but I decided against it because there is probably minimal similarity between all of these music genres. 

In [ ]:
unique = genre_m1.unique()
to_remove = [] 

# genres that have a single instance only will be placed within the to_remove array
for genre in unique:
    if genre_m1.value_counts()[genre] < 20: # 10 was arbitrarily chosen
        to_remove += [genre]
len(to_remove)

Now replacing our original genre columns with our updated version

In [ ]:
df['Top Genre'] = genre_m1
df

In [ ]:
df.set_index(["Top Genre"],drop = False, inplace = True)
for name in to_remove:
    type(name)
    df.drop(index = str(name), inplace = True)
    

In [ ]:
df["Top Genre"].value_counts()

<h2> Model Creation

Since we are dealing with a classification problem, several models will be tried:
1. Random Forest
2. Naive Bayes
3. Stochastic Gradient Descent Classifier
4. Logistic Regression

<h3> Method 1

first preparing the training and testing dataset with proper standardization using StandardScaler

In [ ]:
train_set, test_set = train_test_split(df, test_size = 0.2, random_state = 42)
# training set
X_train = train_set.values[:,1:]
y_train = train_set.values[:,0]

# test set
X_test = test_set.values[:,1:]
y_test = test_set.values[:,0]

In [ ]:
from sklearn.preprocessing import StandardScaler

standard_scaler = StandardScaler().fit(X_train)

# Standard Scaler
X_train_ST = standard_scaler.transform(X_train)
X_test_ST = standard_scaler.transform(X_test)

The labels need to be converted into a form that can be understood by the models, One hot encoding will be used here

In [ ]:
# obtaining all unique classes
unique = np.unique(y_train)

In [ ]:
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import LabelEncoder
# 1 hot encoding
y_test_1hot = label_binarize(y_test, classes = unique)
y_train_1hot = label_binarize(y_train, classes = unique)

# labelling
y_test_label = LabelEncoder()

Now creating the instances of the models

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.multiclass import OneVsOneClassifier

models = []
models += [['Naive Bayes', GaussianNB()]]
models += [['SGD', OneVsOneClassifier(SGDClassifier())]]
models += [['Logistic', LogisticRegression(multi_class = 'ovr')]]
rand_forest = RandomForestClassifier(random_state = 42, min_samples_split = 5)

Now training the models using k cross validation

In [ ]:
result_ST =[]
kfold = StratifiedKFold(n_splits = 10, random_state = 1, shuffle = True)

# Random Forest has to be done separately since it takes in one hot encoded labels instead
RF_cross_val_score = cross_val_score(rand_forest, X_train_ST, y_train_1hot, cv = 10, scoring = 'accuracy')
print('%s: %f (%f)' % ('Random Forest', RF_cross_val_score.mean(), RF_cross_val_score.std()))

for name, model in models:
    cv_score = cross_val_score(model, X_train_ST, y_train, cv = kfold, scoring = 'accuracy')
    result_ST.append(cv_score)
    print('%s: %f (%f)' % (name,cv_score.mean(), cv_score.std()))

All of our models have very low accuracy, which is most likely due to the lack of available data. Let's move forward however, and select a model. While it appears that Logistic Regression has the highest accuracy (56%), let's calculate the respective recall and precision values. Note that micro averaging will be used for the models (w/ exception to Random Forest) bc of the imbalance in class examples.

In [ ]:
from sklearn.metrics import precision_score, recall_score

result_precision_recall = []

# same reasoning as before for Random Forest
y_temp_randforest = cross_val_predict(rand_forest, X_train_ST, y_train_1hot, cv = 10)
result_precision_recall += [['Random Forest', precision_score(y_train_1hot, y_temp_randforest, average = "micro"), 
                            recall_score(y_train_1hot, y_temp_randforest, average = "micro")]]

print('%s| %s: %f, %s (%f)' % ('Random Forest', 'Precision Score: ', precision_score(y_train_1hot, y_temp_randforest, average = "micro"), 
                           'Recall Score: ', recall_score(y_train_1hot, y_temp_randforest, average = "micro")))

for name, model in models:
    y_pred = cross_val_predict(model, X_train_ST, y_train, cv = kfold)
    precision = precision_score(y_train, y_pred, average = "micro")
    recall = recall_score(y_train, y_pred, average = "micro")
    # storing the precision and recall values
    result_precision_recall += [[name , precision, recall]]
    print('%s| %s: %f, %s (%f)' % (name, 'Precision Score: ', precision, 'Recall Score: ', recall))


From the given precision and recall scores, we can now find the respective f1 score and use the highest score to select our model.

In [ ]:
from sklearn.metrics import f1_score

for name, precision, recall in result_precision_recall:
    print("%s: %f" % (name, 2 * (precision * recall) / (precision + recall)))

Given that Logistic Regression has the highest f1 score, we will move forward with that model.

<h2> Evaluation

<h3> Method 1

Our chosen model was logistic regression, so let's evaluate our trained model on the test data set now

In [ ]:
# training the models
model_method1 = LogisticRegression(multi_class = 'ovr').fit(X_train_ST, y_train)

# getting predictions
predictions_method1 = model_method1.predict(X_test_ST)

Now evaluating our accuracy and f1 score

In [ ]:
from sklearn.metrics import confusion_matrix
print(f1_score(y_test, predictions_method1, labels = unique, average = 'micro' ))

This low f1_score is most definitely from the lack of data available

<h2> Data Preparation

<h3> Method 2

In [ ]:
# recalling our original dataframe
df = df_2000.append(df_top10s, ignore_index = True)
attributes = df.columns[1:]
for attribute in attributes:
    temp = df[attribute]
    for instance in range(len(temp)):
        if(type(temp[instance]) == str):
            df[attribute][instance] = float(temp[instance].replace(',',''))
# check data types using the following code
# df.dtypes

In [ ]:
genre = df['Top Genre'].str.split(" ")
genre